## Capstone Project for IBM Data Science Specialization ##

This notebook will be used for the Capstone Project

Student: Jono Pike

In [3]:
# Import packages
import pandas as pd
import numpy as np

print ("Packages loaded")

Packages loaded


In [4]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Week 3 ##
### Segmenting and Clustering Neighborhoods in Toronto ###

#### 1. Import the required libraries / packages ####

In [413]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

# read DataFrame to String
from ast import literal_eval

print ('Packages and libraries imported')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Packages and libraries imported


#### 2: Import and clean up the data ####
##### 2a: Import the data from GitHub #####

In [114]:
# Copied and pasted from the Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M into a .csv, which was uploaded to GitHub

# Define the data location
data_url = "https://raw.githubusercontent.com/Soltis12/coursera_capstone/master/09_Toronto_Neighborhoods.csv"

# Import as a Pandas Dataframe
Toronto_Data = pd.read_csv(data_url)

# View first 5 rows to ensure data loaded correctly
Toronto_Data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


##### 2b. Drop where Borough is not assigned #####

In [115]:
# Drop any rows where Neighbourhood is 'Not assigned'
Toronto_Data = Toronto_Data[Toronto_Data.Borough != 'Not assigned']
Toronto_Data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


##### 2c. Where multiple neighborhoods for the same postcode, concatenate the neighborhoods so postcodes have one row each #####

In [260]:
# Get the unique postcodes and assign a ref number
Toronto_Postcodes = Toronto_Data[['Postcode']] # Get the postcode data
Toronto_Postcodes = Toronto_Postcodes.drop_duplicates() # Drop the duplicate postcodes
Toronto_Postcodes = Toronto_Postcodes.reset_index() # Create a new index
Toronto_Postcodes = Toronto_Postcodes.drop(['index'], axis = 1) # Drop the old index
Toronto_Postcodes['RefNum'] = Toronto_Postcodes.index # Will be used for loop iterations
Toronto_Postcodes.head()

,Postcode,RefNum
0,M3A,0
1,M4A,1
2,M5A,2
3,M6A,3
4,M7A,4


In [336]:
# Identify the max of RefNum to get the number of required iterations
Postcode_Iters = Toronto_Postcodes['RefNum'].max()
Postcode_Iters

102

In [357]:
# Blank DataFrame for the iterations to concatenate onto:

Postcodes_Concat = pd.DataFrame(columns = ['Postcode','Neighborhood'])

# Iteration to get the neighborhoods for each postcode

for i in range(Postcode_Iters):
    Postcode_Select = Toronto_Postcodes[Toronto_Postcodes['RefNum']==i]['Postcode'].values[0]
    N_Hoods = Toronto_Data[Toronto_Data['Postcode']==Postcode_Select]['Neighbourhood']
    N_Hoods = N_Hoods.reset_index() # Create a new index
    N_Hoods = N_Hoods.drop(['index'], axis = 1) # Drop the old index
    N_Hoods['RefNum'] = N_Hoods.index # Will be used for loop iterations, so start from 1
    N_Hood_Iters = N_Hoods['RefNum'].max() + 1

    N_Hood_Append = ''
    
    for i_nh in range(N_Hood_Iters):
        N_Hood_Select_String = '' # Start with blank for each new postcode
        N_Hood_Select = N_Hoods[N_Hoods['RefNum']==i_nh]['Neighbourhood'].values[0]
        N_Hood_Select_String = N_Hood_Select_String + N_Hood_Select
        N_Hood_Append = N_Hood_Append + N_Hood_Select_String
        if i_nh != (N_Hood_Iters-1):
            N_Hood_Append = N_Hood_Append + ', '
        
        N_Hood_Series = ([Postcode_Select, N_Hood_Append])
   
    Postcodes_Concat = Postcodes_Concat.append(pd.Series([Postcode_Select, N_Hood_Append], index=['Postcode','Neighborhood']), ignore_index=True)

Postcodes_Concat

,Postcode,Neighborhood
0,M3A,Parkwoods
1,M4A,Victoria Village
2,M5A,"Harbourfront, Regent Park"
3,M6A,"Lawrence Heights, Lawrence Manor"
4,M7A,Not assigned
5,M9A,Islington Avenue
6,M1B,"Rouge, Malvern"
7,M3B,Don Mills North
8,M4B,"Woodbine Gardens, Parkview Hill"
9,M5B,"Ryerson, Garden District"


In [362]:
# Create a new dataframe to show postcode by borough
Borough_Postcode = Toronto_Data[['Postcode','Borough']]
Borough_Postcode = Borough_Postcode.drop_duplicates()
Borough_Postcode

,Postcode,Borough
2,M3A,North York
3,M4A,North York
4,M5A,Downtown Toronto
6,M6A,North York
8,M7A,Queen's Park
10,M9A,Etobicoke
11,M1B,Scarborough
14,M3B,North York
15,M4B,East York
17,M5B,Downtown Toronto


In [369]:
# Ensure that each postcode has a single borough
Borough_Aggr = Borough_Postcode[['Postcode', 'Borough']].groupby(['Postcode']).agg(['count'])
Borough_Max = Borough_Aggr['Borough'].max()
Borough_Max

count    1
dtype: int64

In [389]:
# Max Borough Count of 1 means performing the join will not cause duplicates.

# Left Join the Borough_Postcode dataframe to the Postcodes_Concat dataframe

Toronto_Data_Refined = pd.merge(Postcodes_Concat, Borough_Postcode, on='Postcode', how='left')
Toronto_Data_Refined = pd.DataFrame(Toronto_Data_Refined)
Toronto_Data_RefinedToronto_Data_Refined.Neighborhood

,Postcode,Neighborhood,Borough
0,M3A,Parkwoods,North York
1,M4A,Victoria Village,North York
2,M5A,"Harbourfront, Regent Park",Downtown Toronto
3,M6A,"Lawrence Heights, Lawrence Manor",North York
4,M7A,Not assigned,Queen's Park
5,M9A,Islington Avenue,Etobicoke
6,M1B,"Rouge, Malvern",Scarborough
7,M3B,Don Mills North,North York
8,M4B,"Woodbine Gardens, Parkview Hill",East York
9,M5B,"Ryerson, Garden District",Downtown Toronto


##### 2d. Where Borough name is populated and Neighborhood name not assigned, replace the Neighborhood name with the Borough

In [390]:
# Where the Neighborhood is 'Not assigned', replace the neighborhood name with the borough name
Toronto_Data_Refined.loc[Toronto_Data_Refined.Neighborhood == 'Not assigned', 'Neighborhood']= Toronto_Data_Refined.Borough
Toronto_Data_Refined

# THIS IS NOW THE CLEANED UP DATA

,Postcode,Neighborhood,Borough
0,M3A,Parkwoods,North York
1,M4A,Victoria Village,North York
2,M5A,"Harbourfront, Regent Park",Downtown Toronto
3,M6A,"Lawrence Heights, Lawrence Manor",North York
4,M7A,Queen's Park,Queen's Park
5,M9A,Islington Avenue,Etobicoke
6,M1B,"Rouge, Malvern",Scarborough
7,M3B,Don Mills North,North York
8,M4B,"Woodbine Gardens, Parkview Hill",East York
9,M5B,"Ryerson, Garden District",Downtown Toronto


#### 2e. Identify the Shape of the DataFrame #####

In [402]:
# Identify Shape of DataFrame
Toronto_Data_Refined.shape

pandas.core.frame.DataFrame

#### 3. Import the co-ordinates data ####

In [401]:
# Co-ordinates data saved to GitHub

# Define the data location
data_url = "https://raw.githubusercontent.com/Soltis12/coursera_capstone/master/09_Geospatial_Coordinates.csv "

# Import as a Pandas Dataframe
Coordinates = pd.read_csv(data_url)

# View first 5 rows to ensure data loaded correctly
Coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [403]:
# Join the co-ordinates to the main table
Coordinates.columns = ['Postcode','Latitude','Longitude'] # Rename Postal Code to Postcode so the column headings are consistent
Toronto_Data_Refined = pd.merge(Toronto_Data_Refined, Coordinates, on='Postcode', how='left')
Toronto_Data_Refined

,Postcode,Neighborhood,Borough,Latitude,Longitude
0,M3A,Parkwoods,North York,43.753259,-79.329656
1,M4A,Victoria Village,North York,43.725882,-79.315572
2,M5A,"Harbourfront, Regent Park",Downtown Toronto,43.654260,-79.360636
3,M6A,"Lawrence Heights, Lawrence Manor",North York,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Islington Avenue,Etobicoke,43.667856,-79.532242
6,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
7,M3B,Don Mills North,North York,43.745906,-79.352188
8,M4B,"Woodbine Gardens, Parkview Hill",East York,43.706397,-79.309937
9,M5B,"Ryerson, Garden District",Downtown Toronto,43.657162,-79.378937


#### 3. Clustering Analysis ####

In [426]:
# Filter where borough name contains word 'Toronto'
Toronto_Data_Sample = Toronto_Data_Refined[Toronto_Data_Refined['Borough'].str.contains("Toronto")]
Toronto_Data_Sample = Toronto_Data_Sample.reset_index() # Create a new index
Toronto_Data_Sample = Toronto_Data_Sample.drop(['index'], axis = 1) # Drop the old index
Toronto_Data_Sample

,Postcode,Neighborhood,Borough,Latitude,Longitude
0,M5A,"Harbourfront, Regent Park",Downtown Toronto,43.654260,-79.360636
1,M5B,"Ryerson, Garden District",Downtown Toronto,43.657162,-79.378937
2,M5C,St. James Town,Downtown Toronto,43.651494,-79.375418
3,M4E,The Beaches,East Toronto,43.676357,-79.293031
4,M5E,Berczy Park,Downtown Toronto,43.644771,-79.373306
5,M5G,Central Bay Street,Downtown Toronto,43.657952,-79.387383
6,M6G,Christie,Downtown Toronto,43.669542,-79.422564
7,M5H,"Adelaide, King, Richmond",Downtown Toronto,43.650571,-79.384568
8,M6H,"Dovercourt Village, Dufferin",West Toronto,43.669005,-79.442259
9,M5J,"Harbourfront East, Toronto Islands, Union Station",Downtown Toronto,43.640816,-79.381752


##### 3a. Create a map of Toronto using the neighborhood data #####

In [427]:
# Identify the public coordinates for Toronto, ON

address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [428]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_Data_Sample['Latitude'], Toronto_Data_Sample['Longitude'], Toronto_Data_Sample['Borough'], Toronto_Data_Sample['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### 3b. Foursquare Credentials (hidden) #####

In [419]:
# The code was removed by Watson Studio for sharing.

##### 3c. First, get information about a single neighborhood #####

In [433]:
# Get a neighborhood
neighborhood_index = 5 # random number within the sample dataset

neighborhood_latitude = Toronto_Data_Sample.loc[neighborhood_index, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_Data_Sample.loc[neighborhood_index, 'Longitude'] # neighborhood longitude value
neighborhood_name = Toronto_Data_Sample.loc[neighborhood_index, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


In [434]:
# Build the Foursquare call
latitude = neighborhood_latitude
longitude = neighborhood_longitude
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

# Call the data
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1a8112f594df03eb8be504'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-537d4d6d498ec171ba22e7fe-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e0931735',
         'name': 'Coffee Shop',
         'pluralName': 'Coffee Shops',
         'primary': True,
         'shortName': 'Coffee Shop'}],
       'id': '537d4d6d498ec171ba22e7fe',
       'location': {'address': '82 Gerrard Street W',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Gerrard & LaPlante',
        'distance': 151,
        'formattedAddress': ['82 Gerrard Street W (Gerrard & LaPlante)',
         'Toronto ON M5G 1Z4',
         'Canada'],
   

In [435]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [439]:
# Transform the JSON extract from Foursquare and put into a Pandas Dataframe: venues_closeby
venues = results['response']['groups'][0]['items']
    
venues_closeby = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues_closeby =venues_closeby.loc[:, filtered_columns]

# filter the category for each row
venues_closeby['venue.categories'] = venues_closeby.apply(get_category_type, axis=1)

# clean columns
venues_closeby.columns = [col.split(".")[-1] for col in venues_closeby.columns]

print('{} venues were returned by Foursquare. First 5 rows below:'.format(venues_closeby.shape[0]))
venues_closeby.head()

80 venues were returned by Foursquare. First 5 rows below:


,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,The Queen and Beaver Public House,Gastropub,43.657472,-79.383524
3,Hailed Coffee,Coffee Shop,43.658833,-79.383684
4,Mercatto,Italian Restaurant,43.660391,-79.387664


##### 3d. Get information about all neighborhoods in sample dataset #####

In [440]:
# Function to retrieve data about nearby venues from all neighborhoods in dataset
def getClosebyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    venues_closeby = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues_closeby.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(venues_closeby)

In [441]:
# Use the function to get the data

names = Toronto_Data_Sample['Neighborhood']
latitudes = Toronto_Data_Sample['Latitude']
longitudes = Toronto_Data_Sample['Longitude']

Toronto_venues = getClosebyVenues(names, latitudes, longitudes, radius=500)

print('{} venues were returned by Foursquare. First 5 rows below:'.format(Toronto_venues.shape[0]))
Toronto_venues.head()

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [442]:
# Get count of venues for each neighborhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,80,80,80,80,80,80
"Chinatown, Grange Park, Kensington Market",97,97,97,97,97,97
Christie,16,16,16,16,16,16


In [443]:
# Identify the number of unique categories
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


In [449]:
# Analyze the neighborhoods to identify venues

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = Toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
Toronto_onehot = Toronto_onehot[cols]

# group the data
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.010000,0.0100,0.000000,0.030000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.00,0.020000,0.010000,0.00,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000,0.000000,0.000000,0.0

##### 3e. Group the data by beighborhood to find most matches per type #####

In [450]:
# Produce a summary of the top 5 frequency of each venue type, per neighborhood

num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.06
2          Restaurant  0.06
3  Italian Restaurant  0.04
4         Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
               venue  freq
0        Coffee Shop  0.14
1               Café  0.10
2     Breakfast Spot  0.10
3  Convenience Store  0.05
4      Grocery Store  0.05


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2         Pizza Place  0.06
3          Comic Shop  0.06
4                Park  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Serv

In [451]:
# Function for converting the summary output into a dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [454]:
num_top_venues = 10
datasource = Toronto_grouped

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = datasource['Neighborhood']

for ind in np.arange(datasource.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(datasource.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Hotel,Gym,Asian Restaurant,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Italian Restaurant,Bakery,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Italian Restaurant,Pet Store,Performing Arts Venue,Climbing Gym,Caribbean Restaurant,Burrito Place
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Park,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Plane,Boutique
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Indian Restaurant,Bakery,Pub,Chinese Restaurant,Pizza Place,Park,Italian Restaurant
6,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Sandwich Place,Bar,Bubble Tea Shop,Indian Restaurant,Falafel Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Gaming Cafe
8,Christie,Grocery Store,Café,Park,Convenience Store,Nightclub,Diner,Baby Store,Athletics & Sports,Restaurant,Italian Restaurant
9,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Pub,Men's Store,Gastropub,Fast Food Restaurant


#### 4. Clustering ####

##### 4a. Create the clusters #####

In [455]:
# set number of clusters
kclusters = 5

grouped_clustering = datasource.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [459]:
merged = Toronto_Data_Sample

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Postcode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,"Harbourfront, Regent Park",Downtown Toronto,43.654260,-79.360636,0,Coffee Shop,Café,Park,Bakery,Pub,Breakfast Spot,Mexican Restaurant,Theater,Health Food Store,Shoe Store
1,M5B,"Ryerson, Garden District",Downtown Toronto,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Sandwich Place,Thai Restaurant,Japanese Restaurant,Pizza Place,Burger Joint
2,M5C,St. James Town,Downtown Toronto,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Cocktail Bar,Bakery,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Gastropub
3,M4E,The Beaches,East Toronto,43.676357,-79.293031,0,Astrologer,Coffee Shop,Pub,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M5E,Berczy Park,Downtown Toronto,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Restaurant,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Italian Restaurant,Bakery,Steakhouse


##### 4b. Visualize the clusters #####

In [460]:
# Visualize the clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### 4c. Understand the Clusters #####

In [462]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront, Regent Park",0,Coffee Shop,Café,Park,Bakery,Pub,Breakfast Spot,Mexican Restaurant,Theater,Health Food Store,Shoe Store
1,"Ryerson, Garden District",0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Sandwich Place,Thai Restaurant,Japanese Restaurant,Pizza Place,Burger Joint
2,St. James Town,0,Coffee Shop,Restaurant,Café,Hotel,Cocktail Bar,Bakery,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Gastropub
3,The Beaches,0,Astrologer,Coffee Shop,Pub,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,Berczy Park,0,Coffee Shop,Cocktail Bar,Restaurant,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Italian Restaurant,Bakery,Steakhouse
5,Central Bay Street,0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Sandwich Place,Bar,Bubble Tea Shop,Indian Restaurant,Falafel Restaurant
6,Christie,0,Grocery Store,Café,Park,Convenience Store,Nightclub,Diner,Baby Store,Athletics & Sports,Restaurant,Italian Restaurant
7,"Adelaide, King, Richmond",0,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Hotel,Gym,Asian Restaurant,Restaurant
8,"Dovercourt Village, Dufferin",0,Supermarket,Pharmacy,Bakery,Park,Art Gallery,Discount Store,Bank,Bar,Middle Eastern Restaurant,Music Venue
9,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Scenic Lookout,Bakery,Restaurant,Brewery,Italian Restaurant


In [463]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Lawrence Park,1,Dim Sum Restaurant,Bus Line,Park,Swim School,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
27,"Runnymede, Swansea",1,Coffee Shop,Pizza Place,Café,Bookstore,Sushi Restaurant,Italian Restaurant,Dessert Shop,Sandwich Place,Burrito Place,Pub


In [464]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North Toronto West,2,Coffee Shop,Sporting Goods Shop,Clothing Store,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant,Rental Car Location


In [465]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,"Moore Park, Summerhill East",3,Playground,Summer Camp,Intersection,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [466]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,"Parkdale, Roncesvalles",4,Breakfast Spot,Gift Shop,Bookstore,Bar,Dog Run,Restaurant,Italian Restaurant,Burger Joint,Bank,Dessert Shop
